In [ ]:
!pip install transformers
!pip install simplet5
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached simplet5-0.1.4.tar.gz (7.3 kB)
  Preparing metadata (setup.py) ... done
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Using cached pytorch_lightning-1.5.10-py3-none-any.whl (527 kB)
  Using cached torchmetrics-0.11.4-py3-none-any.whl (519 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 60.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplet5 0.1.4 requires transformers==4.16.2, but you have transformers 4.29.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5
import tensorflow as tf

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [ ]:
Data = pd.read_excel("/content/Arabic_data.xlsx", nrows=10000)#, delimiter='~'
Data = Data.rename(columns={"Processed Text":"source_text", "summarizer":"target_text"})
Train, Test = train_test_split(Data, test_size=0.2, random_state=42)
Train, Eval = train_test_split(Train, test_size=0.1, random_state=42)

In [ ]:
Path = "/content/Model"
model = SimpleT5()
model.load_model("t5","UBC-NLP/AraT5-base-title-generation") # snrspeaks/t5-one-line-summary
model.train(train_df=Train,
            eval_df=Eval,
            source_max_token_len=512,
            target_max_token_len=50,
            outputdir = Path + "/Output",
            batch_size=4, max_epochs=5, use_gpu=True)
model.model.save_pretrained(Path + "/SavedModel")
model.tokenizer.save_pretrained(Path + "/SavedModel")
X = Test["source_text"].values
Y = Test["target_text"].values
Predict = []
j = 0
for x in X:
  Predict.append(model.predict(x))
  j += 1
  print(str(j)+"/"+str(len(X))+")")
Predict = np.concatenate(Predict)
Predict = np.array(Predict)
Y = np.squeeze(Y)
pd.DataFrame({'Actual': Y, 'Predicted': Predict}).to_csv(Path + "/Predictions.csv", index=False)

In [ ]:
Y = Test["target_text"].values
Y = np.squeeze(Y[:len(Predict)])
pd.DataFrame({'Actual': Y, 'Predicted': Predict}).to_csv(Path + "/Predictions.csv", index=False)

In [ ]:
!pip install rouge
from rouge import Rouge
import pandas as pd
df = pd.read_csv("/content/Model/Predictions.csv")
hypotheses = df['Predicted'].values
references = df['Actual'].values
# Create a Rouge object
rouge = Rouge()
# Calculate ROUGE scores
scores = rouge.get_scores(hypotheses, references, avg=True)
# Print the scores
print(scores)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
{'rouge-1': {'r': 0.43573451276577335, 'p': 0.5765565194301157, 'f': 0.4755773966053609}, 'rouge-2': {'r': 0.2990145435983571, 'p': 0.4054847884273707, 'f': 0.3273213911992807}, 'rouge-l': {'r': 0.4271476344310562, 'p': 0.5659264666101808, 'f': 0.4665039154837726}}
